# Yeni Notebook

Bu notebook yeni bir analiz için oluşturulmuştur.


In [ ]:
# Kod hücresi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)


## 📂 Veri Setini Yükleme


In [11]:
# Veri setini yükle
from pathlib import Path

# Veri dosyasının yolu
data_path = Path("data/online_retail_II.xlsx")

# Veriyi yükle
print(f"📂 Veri dosyası yükleniyor: {data_path}")
print(f"📂 Dosya var mı: {data_path.exists()}")

if not data_path.exists():
    # Alternatif yol dene
    alt_yol = Path.cwd() / "data" / "online_retail_II.xlsx"
    if alt_yol.exists():
        data_path = alt_yol
        print(f"📂 Alternatif yol kullanılıyor: {data_path}")
    else:
        raise FileNotFoundError(f"Veri dosyası bulunamadı! Denenen yollar:\n1. {Path('data/online_retail_II.xlsx')}\n2. {alt_yol}")

df = pd.read_excel(data_path)

print(f"✅ Veri başarıyla yüklendi!")
print(f"📊 Veri boyutu: {df.shape}")
print(f"\n📋 İlk 5 satır:")
df.head()


📂 Veri dosyası yükleniyor: data\online_retail_II.xlsx
📂 Dosya var mı: True
✅ Veri başarıyla yüklendi!
📊 Veri boyutu: (525461, 8)

📋 İlk 5 satır:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [12]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,525461.000000,525461,525461.000000,417534.000000
mean,10.337667,2010-06-28 11:37:36.845017856,4.688834,15360.645478
min,-9600.000000,2009-12-01 07:45:00,-53594.360000,12346.000000
25%,1.000000,2010-03-21 12:20:00,1.250000,13983.000000
50%,3.000000,2010-07-06 09:51:00,2.100000,15311.000000
75%,10.000000,2010-10-15 12:45:00,4.210000,16799.000000
max,19152.000000,2010-12-09 20:01:00,25111.090000,18287.000000
std,107.424110,NaN,146.126914,1680.811316


In [13]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [14]:
## 🗑️ Sütunları Atma

df = df.drop(columns=['Invoice', 'Description'])
print(f"Yeni boyut: {df.shape}")
df.head()


Yeni boyut: (525461, 6)


,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country
0,85048,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,79323P,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,79323W,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,22041,48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,21232,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [15]:
## 📊 StockCode Analizi

print(f"Benzersiz StockCode: {df['StockCode'].nunique():,}")
print(f"\nTop 10 StockCode:")
df['StockCode'].value_counts().head(10)


Benzersiz StockCode: 4,632

Top 10 StockCode:


StockCode
85123A    3516
22423     2221
85099B    2057
21212     1933
21232     1843
20725     1620
84879     1458
84991     1400
21754     1386
20914     1276
Name: count, dtype: int64

In [16]:
## 💡 StockCode İşleme - Önerilen Yaklaşım

# Frequency Encoding (popülerlik bilgisi)
df['StockCode_Freq'] = df['StockCode'].map(df['StockCode'].value_counts())

# StockCode bazlı özellikler
df['StockCode_AvgQty'] = df.groupby('StockCode')['Quantity'].transform('mean')
df['StockCode_AvgPrice'] = df.groupby('StockCode')['Price'].transform('mean')

# Orijinal StockCode'u kaldır
df = df.drop(columns=['StockCode'])

print(f"✅ StockCode işlendi - Yeni boyut: {df.shape}")
df.head()


✅ StockCode işlendi - Yeni boyut: (525461, 8)


,Quantity,InvoiceDate,Price,Customer ID,Country,StockCode_Freq,StockCode_AvgQty,StockCode_AvgPrice
0,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,319,4.197492,8.999310
1,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,350,3.925714,7.029343
2,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,476,3.756303,6.821744
3,48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,213,16.352113,3.303333
4,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,1843,14.412914,1.444086


In [17]:
## 🌍 Country Analizi

print(f"Benzersiz ülke sayısı: {df['Country'].nunique()}")
print(f"\nTop 10 ülke:")
df['Country'].value_counts().head(10)


Benzersiz ülke sayısı: 40

Top 10 ülke:


Country
United Kingdom     485852
EIRE                 9670
Germany              8129
France               5772
Netherlands          2769
Spain                1278
Switzerland          1187
Portugal             1101
Belgium              1054
Channel Islands       906
Name: count, dtype: int64

In [25]:
## 💡 Country One-Hot Encoding (0/1 Değerleri, Sütunlar En Sonda)

# Country sütunu varsa işle
if 'Country' in df.columns:
    # One-Hot Encoding (dtype=int ile 0/1 değerleri)
    country_dummies = pd.get_dummies(df['Country'], prefix='Country', dtype=int)
    
    # Orijinal Country sütununu kaldır
    df = df.drop(columns=['Country'])
    
    # Country sütunlarını en sona ekle
    df = pd.concat([df, country_dummies], axis=1)
    
    print(f"✅ Country One-Hot Encoding yapıldı (0/1 değerleri) - Yeni boyut: {df.shape}")
    print(f"\n📋 Sütun sırası (Country sütunları en sonda):")
    print(df.columns.tolist())
else:
    print("⚠️ Country sütunu zaten işlenmiş veya bulunamadı!")
    print(f"Mevcut sütunlar: {df.columns.tolist()}")

df.head()


⚠️ Country sütunu zaten işlenmiş veya bulunamadı!
Mevcut sütunlar: ['Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'StockCode_Freq', 'StockCode_AvgQty', 'StockCode_AvgPrice', 'Country_Australia', 'Country_Austria', 'Country_Bahrain', 'Country_Belgium', 'Country_Bermuda', 'Country_Brazil', 'Country_Canada', 'Country_Channel Islands', 'Country_Cyprus', 'Country_Denmark', 'Country_EIRE', 'Country_Finland', 'Country_France', 'Country_Germany', 'Country_Greece', 'Country_Hong Kong', 'Country_Iceland', 'Country_Israel', 'Country_Italy', 'Country_Japan', 'Country_Korea', 'Country_Lebanon', 'Country_Lithuania', 'Country_Malta', 'Country_Netherlands', 'Country_Nigeria', 'Country_Norway', 'Country_Poland', 'Country_Portugal', 'Country_RSA', 'Country_Singapore', 'Country_Spain', 'Country_Sweden', 'Country_Switzerland', 'Country_Thailand', 'Country_USA', 'Country_United Arab Emirates', 'Country_United Kingdom', 'Country_Unspecified', 'Country_West Indies']


,Quantity,InvoiceDate,Price,Customer ID,StockCode_Freq,StockCode_AvgQty,StockCode_AvgPrice,Country_Australia,Country_Austria,Country_Bahrain,...,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_Thailand,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,Country_West Indies
0,12,2009-12-01 07:45:00,6.95,13085.0,319,4.197492,8.999310,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,12,2009-12-01 07:45:00,6.75,13085.0,350,3.925714,7.029343,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,12,2009-12-01 07:45:00,6.75,13085.0,476,3.756303,6.821744,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,48,2009-12-01 07:45:00,2.10,13085.0,213,16.352113,3.303333,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,24,2009-12-01 07:45:00,1.25,13085.0,1843,14.412914,1.444086,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [26]:
## 🔄 Unspecified'i En Çok Görülen Ülkeye Ata ve Sil

# Country sütunlarını bul
country_cols = [col for col in df.columns if col.startswith('Country_')]

if 'Country_Unspecified' in country_cols:
    # En çok görülen ülkeyi bul (Country_Unspecified hariç)
    country_cols_without_unspecified = [col for col in country_cols if col != 'Country_Unspecified']
    country_totals = df[country_cols_without_unspecified].sum()
    most_common_country = country_totals.idxmax()
    
    print(f"📊 En çok görülen ülke: {most_common_country}")
    print(f"   Toplam satır sayısı: {country_totals[most_common_country]:,}")
    
    # Country_Unspecified = 1 olan satırları bul
    unspecified_mask = df['Country_Unspecified'] == 1
    unspecified_count = unspecified_mask.sum()
    
    print(f"\n🔄 İşlem:")
    print(f"   - Country_Unspecified = 1 olan satır sayısı: {unspecified_count:,}")
    
    # Bu satırları en çok görülen ülkeye ata
    df.loc[unspecified_mask, most_common_country] = 1
    df.loc[unspecified_mask, 'Country_Unspecified'] = 0
    
    # Country_Unspecified sütununu sil
    df = df.drop(columns=['Country_Unspecified'])
    
    print(f"   ✅ {unspecified_count:,} satır {most_common_country} olarak güncellendi")
    print(f"   ✅ Country_Unspecified sütunu silindi")
    print(f"\n📊 Yeni boyut: {df.shape}")
else:
    print("⚠️ Country_Unspecified sütunu bulunamadı!")


📊 En çok görülen ülke: Country_United Kingdom
   Toplam satır sayısı: 485,852

🔄 İşlem:
   - Country_Unspecified = 1 olan satır sayısı: 310
   ✅ 310 satır Country_United Kingdom olarak güncellendi
   ✅ Country_Unspecified sütunu silindi

📊 Yeni boyut: (525461, 46)


In [27]:
df.head(10)

,Quantity,InvoiceDate,Price,Customer ID,StockCode_Freq,StockCode_AvgQty,StockCode_AvgPrice,Country_Australia,Country_Austria,Country_Bahrain,...,Country_RSA,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_Thailand,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_West Indies
0,12,2009-12-01 07:45:00,6.95,13085.0,319,4.197492,8.999310,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,12,2009-12-01 07:45:00,6.75,13085.0,350,3.925714,7.029343,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,12,2009-12-01 07:45:00,6.75,13085.0,476,3.756303,6.821744,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,48,2009-12-01 07:45:00,2.10,13085.0,213,16.352113,3.303333,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,24,2009-12-01 07:45:00,1.25,13085.0,1843,14.412914,1.444086,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,24,2009-12-01 07:45:00,1.65,13085.0,508,9.596457,2.116102,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,24,2009-12-01 07:45:00,1.25,13085.0,478,12.663180,1.640314,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,10,2009-12-01 07:45:00,5.95,13085.0,655,5.871756,8.369221,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,12,2009-12-01 07:46:00,2.55,13085.0,226,6.420354,3.058894,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,12,2009-12-01 07:46:00,3.75,13085.0,251,4.784861,4.120558,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [28]:
## 👤 Customer ID Analizi

print(f"Benzersiz Customer ID sayısı: {df['Customer ID'].nunique():,}")
print(f"Toplam satır sayısı: {len(df):,}")
print(f"Eksik değer sayısı: {df['Customer ID'].isnull().sum():,}")
print(f"Eksik değer yüzdesi: {(df['Customer ID'].isnull().sum()/len(df)*100):.2f}%")
print(f"\nTop 10 Customer ID:")
df['Customer ID'].value_counts().head(10)


Benzersiz Customer ID sayısı: 4,383
Toplam satır sayısı: 525,461
Eksik değer sayısı: 107,927
Eksik değer yüzdesi: 20.54%

Top 10 Customer ID:


Customer ID
14911.0    5710
17841.0    5114
14606.0    3927
14156.0    2710
12748.0    2665
17850.0    2515
16549.0    2274
15311.0    2226
14527.0    1826
14646.0    1805
Name: count, dtype: int64

In [31]:
## ✅ Customer ID İşleme - Özellik Mühendisliği

# Customer ID bazlı özellikler çıkar (StockCode'a benzer)
# Eksik değerler için: Tüm özellikler 0 olacak

if 'Customer ID' in df.columns:
    # Frequency Encoding
    df['CustomerID_Freq'] = df['Customer ID'].map(df['Customer ID'].value_counts())
    
    # Customer ID bazlı özellikler
    df['CustomerID_AvgQty'] = df.groupby('Customer ID')['Quantity'].transform('mean')
    df['CustomerID_AvgPrice'] = df.groupby('Customer ID')['Price'].transform('mean')
    df['CustomerID_TotalSpent'] = df.groupby('Customer ID')['Price'].transform('sum')
    
    # Eksik değerler için 0 değeri ata
    df['CustomerID_Freq'] = df['CustomerID_Freq'].fillna(0)
    df['CustomerID_AvgQty'] = df['CustomerID_AvgQty'].fillna(0)
    df['CustomerID_AvgPrice'] = df['CustomerID_AvgPrice'].fillna(0)
    df['CustomerID_TotalSpent'] = df['CustomerID_TotalSpent'].fillna(0)
    
    # Customer ID'yi kaldır
    df = df.drop(columns=['Customer ID'])
    
    print(f"✅ Customer ID işlendi - Yeni boyut: {df.shape}")
    print(f"\n📋 Yeni özellikler:")
    print("   - CustomerID_Freq: Toplam sipariş sayısı")
    print("   - CustomerID_AvgQty: Ortalama sipariş miktarı")
    print("   - CustomerID_AvgPrice: Ortalama fiyat")
    print("   - CustomerID_TotalSpent: Toplam harcama")
    print(f"\n💡 Eksik değerler 0 olarak işlendi (bilinmeyen müşteri)")
else:
    print("⚠️ Customer ID sütunu bulunamadı veya zaten işlenmiş!")

df.head()


⚠️ Customer ID sütunu bulunamadı veya zaten işlenmiş!


,Quantity,InvoiceDate,Price,StockCode_Freq,StockCode_AvgQty,StockCode_AvgPrice,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,...,Country_Switzerland,Country_Thailand,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_West Indies,CustomerID_Freq,CustomerID_AvgQty,CustomerID_AvgPrice,CustomerID_TotalSpent
0,12,2009-12-01 07:45:00,6.95,319,4.197492,8.999310,0,0,0,0,...,0,0,0,0,1,0,63.0,11.539683,16.526508,1041.17
1,12,2009-12-01 07:45:00,6.75,350,3.925714,7.029343,0,0,0,0,...,0,0,0,0,1,0,63.0,11.539683,16.526508,1041.17
2,12,2009-12-01 07:45:00,6.75,476,3.756303,6.821744,0,0,0,0,...,0,0,0,0,1,0,63.0,11.539683,16.526508,1041.17
3,48,2009-12-01 07:45:00,2.10,213,16.352113,3.303333,0,0,0,0,...,0,0,0,0,1,0,63.0,11.539683,16.526508,1041.17
4,24,2009-12-01 07:45:00,1.25,1843,14.412914,1.444086,0,0,0,0,...,0,0,0,0,1,0,63.0,11.539683,16.526508,1041.17


In [32]:
## 📅 InvoiceDate Analizi

if 'InvoiceDate' in df.columns:
    print(f"Tarih aralığı: {df['InvoiceDate'].min()} - {df['InvoiceDate'].max()}")
    print(f"Eksik değer: {df['InvoiceDate'].isnull().sum()}")
    print(f"\nÖrnek değerler:")
    print(df['InvoiceDate'].head())
else:
    print("⚠️ InvoiceDate sütunu bulunamadı!")


Tarih aralığı: 2009-12-01 07:45:00 - 2010-12-09 20:01:00
Eksik değer: 0

Örnek değerler:
0   2009-12-01 07:45:00
1   2009-12-01 07:45:00
2   2009-12-01 07:45:00
3   2009-12-01 07:45:00
4   2009-12-01 07:45:00
Name: InvoiceDate, dtype: datetime64[ns]


In [33]:
## 💡 InvoiceDate İşlemleri - Seçenekler

print("InvoiceDate İçin Yapılabilecekler:")
print("=" * 70)

print("\n1️⃣ ZAMAN BAZLI ÖZELLİKLER ÇIKARMAK (ÖNERİLEN):")
print("   - Yıl, Ay, Gün")
print("   - Haftanın günü (0=Pazartesi, 6=Pazar)")
print("   - Saat (0-23)")
print("   - Hafta numarası")
print("   - Mevsim (1-4)")
print("   - İş günü mü? (Hafta içi/Hafta sonu)")
print("   ✅ Model için çok yararlı")
print("   ✅ Mevsimsel ve günlük trendleri yakalar")

print("\n2️⃣ TARİHİ SAYISAL DEĞERE DÖNÜŞTÜRMEK:")
print("   - Timestamp (Unix zaman damgası)")
print("   - Gün sayısı (başlangıç tarihinden itibaren)")
print("   ⚠️ Daha az bilgi verir")

print("\n3️⃣ TARİHİ KALDIRMAK:")
print("   - Eğer zaman bilgisi önemli değilse")
print("   ❌ Mevsimsel trendleri kaybedersiniz")

print("\n" + "=" * 70)
print("💡 ÖNERİ: Zaman bazlı özellikler çıkarmak")
print("   - Satışların günlük, haftalık, aylık trendlerini yakalar")


InvoiceDate İçin Yapılabilecekler:

1️⃣ ZAMAN BAZLI ÖZELLİKLER ÇIKARMAK (ÖNERİLEN):
   - Yıl, Ay, Gün
   - Haftanın günü (0=Pazartesi, 6=Pazar)
   - Saat (0-23)
   - Hafta numarası
   - Mevsim (1-4)
   - İş günü mü? (Hafta içi/Hafta sonu)
   ✅ Model için çok yararlı
   ✅ Mevsimsel ve günlük trendleri yakalar

2️⃣ TARİHİ SAYISAL DEĞERE DÖNÜŞTÜRMEK:
   - Timestamp (Unix zaman damgası)
   - Gün sayısı (başlangıç tarihinden itibaren)
   ⚠️ Daha az bilgi verir

3️⃣ TARİHİ KALDIRMAK:
   - Eğer zaman bilgisi önemli değilse
   ❌ Mevsimsel trendleri kaybedersiniz

💡 ÖNERİ: Zaman bazlı özellikler çıkarmak
   - Satışların günlük, haftalık, aylık trendlerini yakalar


In [34]:
## ✅ InvoiceDate İşleme - Zaman Bazlı Özellikler

if 'InvoiceDate' in df.columns:
    # Zaman bazlı özellikler çıkar
    df['Year'] = df['InvoiceDate'].dt.year
    df['Month'] = df['InvoiceDate'].dt.month
    df['Day'] = df['InvoiceDate'].dt.day
    df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek  # 0=Pazartesi, 6=Pazar
    df['Hour'] = df['InvoiceDate'].dt.hour
    df['WeekOfYear'] = df['InvoiceDate'].dt.isocalendar().week
    df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)  # Cumartesi ve Pazar
    
    # Mevsim (1=Kış, 2=İlkbahar, 3=Yaz, 4=Sonbahar)
    df['Season'] = df['Month'].apply(lambda x: 1 if x in [12,1,2] else 2 if x in [3,4,5] else 3 if x in [6,7,8] else 4)
    
    # Orijinal InvoiceDate'i kaldır
    df = df.drop(columns=['InvoiceDate'])
    
    print(f"✅ InvoiceDate işlendi - Yeni boyut: {df.shape}")
    print(f"\n📋 Oluşturulan özellikler:")
    print("   ✅ Year: Yıl")
    print("   ✅ Month: Ay (1-12)")
    print("   ✅ Day: Gün (1-31)")
    print("   ✅ DayOfWeek: Haftanın günü (0=Pazartesi, 6=Pazar)")
    print("   ✅ Hour: Saat (0-23)")
    print("   ✅ WeekOfYear: Hafta numarası")
    print("   ✅ IsWeekend: Hafta sonu mu? (0=Hayır, 1=Evet)")
    print("   ✅ Season: Mevsim (1=Kış, 2=İlkbahar, 3=Yaz, 4=Sonbahar)")
else:
    print("⚠️ InvoiceDate sütunu bulunamadı!")

df.head()


✅ InvoiceDate işlendi - Yeni boyut: (525461, 56)

📋 Oluşturulan özellikler:
   ✅ Year: Yıl
   ✅ Month: Ay (1-12)
   ✅ Day: Gün (1-31)
   ✅ DayOfWeek: Haftanın günü (0=Pazartesi, 6=Pazar)
   ✅ Hour: Saat (0-23)
   ✅ WeekOfYear: Hafta numarası
   ✅ IsWeekend: Hafta sonu mu? (0=Hayır, 1=Evet)
   ✅ Season: Mevsim (1=Kış, 2=İlkbahar, 3=Yaz, 4=Sonbahar)


,Quantity,Price,StockCode_Freq,StockCode_AvgQty,StockCode_AvgPrice,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,Country_Bermuda,...,CustomerID_AvgPrice,CustomerID_TotalSpent,Year,Month,Day,DayOfWeek,Hour,WeekOfYear,IsWeekend,Season
0,12,6.95,319,4.197492,8.999310,0,0,0,0,0,...,16.526508,1041.17,2009,12,1,1,7,49,0,1
1,12,6.75,350,3.925714,7.029343,0,0,0,0,0,...,16.526508,1041.17,2009,12,1,1,7,49,0,1
2,12,6.75,476,3.756303,6.821744,0,0,0,0,0,...,16.526508,1041.17,2009,12,1,1,7,49,0,1
3,48,2.10,213,16.352113,3.303333,0,0,0,0,0,...,16.526508,1041.17,2009,12,1,1,7,49,0,1
4,24,1.25,1843,14.412914,1.444086,0,0,0,0,0,...,16.526508,1041.17,2009,12,1,1,7,49,0,1


In [35]:
## 🔀 Veri Setini Karıştırma

# Veri setini rastgele karıştır (shuffle)
# random_state ile tekrarlanabilirlik sağlanır
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ Veri seti karıştırıldı - Boyut: {df.shape}")
print(f"\n📊 İlk 5 satır (karıştırılmış):")
df.head()


✅ Veri seti karıştırıldı - Boyut: (525461, 56)

📊 İlk 5 satır (karıştırılmış):


,Quantity,Price,StockCode_Freq,StockCode_AvgQty,StockCode_AvgPrice,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,Country_Bermuda,...,CustomerID_AvgPrice,CustomerID_TotalSpent,Year,Month,Day,DayOfWeek,Hour,WeekOfYear,IsWeekend,Season
0,6,7.95,559,3.604651,8.133989,0,0,0,0,0,...,1.861538,48.40,2010,10,14,3,12,41,0,4
1,3,2.10,615,8.330081,2.565626,0,0,0,0,0,...,2.086904,4745.62,2010,11,7,6,16,44,1,4
2,48,0.55,223,16.116592,0.676861,0,0,0,0,0,...,2.455732,201.37,2010,10,3,6,14,39,1,4
3,12,1.25,418,36.064593,1.350431,0,0,0,0,0,...,1.846506,153.26,2010,6,11,4,11,23,0,3
4,12,4.95,178,5.696629,4.914719,0,0,0,0,0,...,4.304452,3219.73,2009,12,20,6,10,51,1,1


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 56 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Quantity                      525461 non-null  int64  
 1   Price                         525461 non-null  float64
 2   StockCode_Freq                525461 non-null  int64  
 3   StockCode_AvgQty              525461 non-null  float64
 4   StockCode_AvgPrice            525461 non-null  float64
 5   Country_Australia             525461 non-null  int64  
 6   Country_Austria               525461 non-null  int64  
 7   Country_Bahrain               525461 non-null  int64  
 8   Country_Belgium               525461 non-null  int64  
 9   Country_Bermuda               525461 non-null  int64  
 10  Country_Brazil                525461 non-null  int64  
 11  Country_Canada                525461 non-null  int64  
 12  Country_Channel Islands       525461 non-nul

In [42]:
## 📊 Outlier Özeti - Çok Outlier Var mı?

# Sayısal sütunları seç (Quantity hariç)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Quantity' in numeric_cols:
    numeric_cols.remove('Quantity')

# Her sütun için outlier sayısını hesapla
outlier_summary = []
total_outliers = 0

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    outlier_count = len(outliers)
    outlier_pct = (outlier_count / len(df)) * 100
    total_outliers += outlier_count
    
    outlier_summary.append({
        'Sütun': col,
        'Outlier_Sayısı': outlier_count,
        'Yüzde': outlier_pct
    })

# Özet DataFrame
outlier_df = pd.DataFrame(outlier_summary).sort_values('Outlier_Sayısı', ascending=False)

print("=" * 70)
print("📊 OUTLIER ÖZETİ")
print("=" * 70)
print(f"\nToplam sütun sayısı: {len(numeric_cols)}")
print(f"Toplam outlier sayısı: {total_outliers:,}")
print(f"Ortalama outlier yüzdesi: {(total_outliers / (len(df) * len(numeric_cols)) * 100):.2f}%")

print(f"\n🔝 EN ÇOK OUTLIER OLAN 10 SÜTUN:")
print(outlier_df.head(10).to_string(index=False))

# Genel değerlendirme
high_outlier_cols = outlier_df[outlier_df['Yüzde'] > 10]
print(f"\n⚠️ %10'dan fazla outlier olan sütun sayısı: {len(high_outlier_cols)}")

if len(high_outlier_cols) > 0:
    print(f"\n📋 Bu sütunlar:")
    for _, row in high_outlier_cols.iterrows():
        print(f"   - {row['Sütun']}: {row['Yüzde']:.2f}% outlier")

print("\n" + "=" * 70)
if len(high_outlier_cols) > len(numeric_cols) * 0.3:
    print("🔴 SONUÇ: Veri setinde ÇOK FAZLA outlier var!")
    print("   💡 Öneri: RobustScaler kullanın veya outlier'ları temizleyin")
elif len(high_outlier_cols) > 0:
    print("🟡 SONUÇ: Veri setinde ORTA DÜZEYDE outlier var")
    print("   💡 Öneri: RobustScaler kullanabilirsiniz")
else:
    print("🟢 SONUÇ: Veri setinde AZ outlier var")
    print("   💡 Öneri: StandardScaler kullanabilirsiniz")
print("=" * 70)


📊 OUTLIER ÖZETİ

Toplam sütun sayısı: 55
Toplam outlier sayısı: 487,450
Ortalama outlier yüzdesi: 1.69%

🔝 EN ÇOK OUTLIER OLAN 10 SÜTUN:
                 Sütun  Outlier_Sayısı     Yüzde
             IsWeekend           75283 14.327039
 CustomerID_TotalSpent           53611 10.202660
       CustomerID_Freq           52427  9.977334
                  Year           45228  8.607299
    StockCode_AvgPrice           44299  8.430502
Country_United Kingdom           39299  7.478957
                 Price           35273  6.712772
      StockCode_AvgQty           34770  6.617047
     CustomerID_AvgQty           28103  5.348256
   CustomerID_AvgPrice           21148  4.024656

⚠️ %10'dan fazla outlier olan sütun sayısı: 2

📋 Bu sütunlar:
   - IsWeekend: 14.33% outlier
   - CustomerID_TotalSpent: 10.20% outlier

🟡 SONUÇ: Veri setinde ORTA DÜZEYDE outlier var
   💡 Öneri: RobustScaler kullanabilirsiniz


In [49]:
## ✅ Train/Test Split

from sklearn.model_selection import train_test_split

# X ve y'yi ayır
X = df.drop(columns=['Quantity'])
y = df['Quantity']

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✅ Train/Test Split yapıldı")
print(f"Train: {X_train.shape[0]:,} satır ({len(X_train)/len(df)*100:.1f}%)")
print(f"Test: {X_test.shape[0]:,} satır ({len(X_test)/len(df)*100:.1f}%)")


✅ Train/Test Split yapıldı
Train: 420,368 satır (80.0%)
Test: 105,093 satır (20.0%)


In [51]:
## 🤖 Model Eğitimi ve Hyperparameter Tuning

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Cross-validation ayarları
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Sonuçları saklamak için
results = []

print("=" * 70)
print("🚀 MODEL EĞİTİMİ VE HYPERPARAMETER TUNING")
print("=" * 70)


🚀 MODEL EĞİTİMİ VE HYPERPARAMETER TUNING


In [52]:
## 1️⃣ Linear Regression

print("\n1️⃣ LINEAR REGRESSION")
print("-" * 70)

lr = LinearRegression()
lr.fit(X_train, y_train)

# Cross-validation
cv_scores = cross_val_score(lr, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)

# Test seti performansı
y_pred = lr.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

results.append({
    'Model': 'Linear Regression',
    'CV_RMSE_Mean': cv_rmse.mean(),
    'CV_RMSE_Std': cv_rmse.std(),
    'Test_RMSE': test_rmse,
    'Test_MAE': test_mae,
    'Test_R2': test_r2,
    'Best_Params': 'N/A (no hyperparameters)'
})

print(f"✅ CV RMSE: {cv_rmse.mean():.4f} (+/- {cv_rmse.std()*2:.4f})")
print(f"✅ Test RMSE: {test_rmse:.4f}")
print(f"✅ Test MAE: {test_mae:.4f}")
print(f"✅ Test R²: {test_r2:.4f}")



1️⃣ LINEAR REGRESSION
----------------------------------------------------------------------
✅ CV RMSE: 91.7475 (+/- 21.9150)
✅ Test RMSE: 87.7021
✅ Test MAE: 10.6168
✅ Test R²: 0.2456


In [53]:
## 2️⃣ Ridge Regression - Hyperparameter Tuning

print("\n2️⃣ RIDGE REGRESSION")
print("-" * 70)

ridge_params = {'alpha': [0.1, 1, 10, 100, 1000]}
ridge = Ridge()

ridge_grid = GridSearchCV(ridge, ridge_params, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
ridge_grid.fit(X_train, y_train)

best_ridge = ridge_grid.best_estimator_
cv_rmse = np.sqrt(-ridge_grid.best_score_)

y_pred = best_ridge.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

results.append({
    'Model': 'Ridge Regression',
    'CV_RMSE_Mean': cv_rmse,
    'CV_RMSE_Std': 0,
    'Test_RMSE': test_rmse,
    'Test_MAE': test_mae,
    'Test_R2': test_r2,
    'Best_Params': f"alpha={ridge_grid.best_params_['alpha']}"
})

print(f"✅ Best Params: {ridge_grid.best_params_}")
print(f"✅ CV RMSE: {cv_rmse:.4f}")
print(f"✅ Test RMSE: {test_rmse:.4f}")
print(f"✅ Test MAE: {test_mae:.4f}")
print(f"✅ Test R²: {test_r2:.4f}")



2️⃣ RIDGE REGRESSION
----------------------------------------------------------------------
✅ Best Params: {'alpha': 1000}
✅ CV RMSE: 92.3987
✅ Test RMSE: 87.6855
✅ Test MAE: 10.5883
✅ Test R²: 0.2459


In [54]:
## 3️⃣ Lasso Regression - Hyperparameter Tuning

print("\n3️⃣ LASSO REGRESSION")
print("-" * 70)

lasso_params = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
lasso = Lasso(max_iter=10000)

lasso_grid = GridSearchCV(lasso, lasso_params, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
lasso_grid.fit(X_train, y_train)

best_lasso = lasso_grid.best_estimator_
cv_rmse = np.sqrt(-lasso_grid.best_score_)

y_pred = best_lasso.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

results.append({
    'Model': 'Lasso Regression',
    'CV_RMSE_Mean': cv_rmse,
    'CV_RMSE_Std': 0,
    'Test_RMSE': test_rmse,
    'Test_MAE': test_mae,
    'Test_R2': test_r2,
    'Best_Params': f"alpha={lasso_grid.best_params_['alpha']}"
})

print(f"✅ Best Params: {lasso_grid.best_params_}")
print(f"✅ CV RMSE: {cv_rmse:.4f}")
print(f"✅ Test RMSE: {test_rmse:.4f}")
print(f"✅ Test MAE: {test_mae:.4f}")
print(f"✅ Test R²: {test_r2:.4f}")



3️⃣ LASSO REGRESSION
----------------------------------------------------------------------
✅ Best Params: {'alpha': 0.1}
✅ CV RMSE: 92.3942
✅ Test RMSE: 87.6794
✅ Test MAE: 10.5903
✅ Test R²: 0.2460


In [55]:
## 4️⃣ Random Forest - Hyperparameter Tuning

print("\n4️⃣ RANDOM FOREST")
print("-" * 70)

rf_params = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5]
}
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

rf_grid = GridSearchCV(rf, rf_params, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
rf_grid.fit(X_train, y_train)

best_rf = rf_grid.best_estimator_
cv_rmse = np.sqrt(-rf_grid.best_score_)

y_pred = best_rf.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

results.append({
    'Model': 'Random Forest',
    'CV_RMSE_Mean': cv_rmse,
    'CV_RMSE_Std': 0,
    'Test_RMSE': test_rmse,
    'Test_MAE': test_mae,
    'Test_R2': test_r2,
    'Best_Params': str(rf_grid.best_params_)
})

print(f"✅ Best Params: {rf_grid.best_params_}")
print(f"✅ CV RMSE: {cv_rmse:.4f}")
print(f"✅ Test RMSE: {test_rmse:.4f}")
print(f"✅ Test MAE: {test_mae:.4f}")
print(f"✅ Test R²: {test_r2:.4f}")



4️⃣ RANDOM FOREST
----------------------------------------------------------------------
Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 

In [ ]:
## 5️⃣ Gradient Boosting - Hyperparameter Tuning

print("\n5️⃣ GRADIENT BOOSTING")
print("-" * 70)

gb_params = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [50, 100],
    'max_depth': [3, 5]
}
gb = GradientBoostingRegressor(random_state=42)

gb_grid = GridSearchCV(gb, gb_params, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
gb_grid.fit(X_train, y_train)

best_gb = gb_grid.best_estimator_
cv_rmse = np.sqrt(-gb_grid.best_score_)

y_pred = best_gb.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

results.append({
    'Model': 'Gradient Boosting',
    'CV_RMSE_Mean': cv_rmse,
    'CV_RMSE_Std': 0,
    'Test_RMSE': test_rmse,
    'Test_MAE': test_mae,
    'Test_R2': test_r2,
    'Best_Params': str(gb_grid.best_params_)
})

print(f"✅ Best Params: {gb_grid.best_params_}")
print(f"✅ CV RMSE: {cv_rmse:.4f}")
print(f"✅ Test RMSE: {test_rmse:.4f}")
print(f"✅ Test MAE: {test_mae:.4f}")
print(f"✅ Test R²: {test_r2:.4f}")


In [ ]:
## 6️⃣ XGBoost - Hyperparameter Tuning

print("\n6️⃣ XGBOOST")
print("-" * 70)

xgb_params = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [50, 100],
    'max_depth': [3, 5]
}
xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
xgb_grid.fit(X_train, y_train)

best_xgb = xgb_grid.best_estimator_
cv_rmse = np.sqrt(-xgb_grid.best_score_)

y_pred = best_xgb.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

results.append({
    'Model': 'XGBoost',
    'CV_RMSE_Mean': cv_rmse,
    'CV_RMSE_Std': 0,
    'Test_RMSE': test_rmse,
    'Test_MAE': test_mae,
    'Test_R2': test_r2,
    'Best_Params': str(xgb_grid.best_params_)
})

print(f"✅ Best Params: {xgb_grid.best_params_}")
print(f"✅ CV RMSE: {cv_rmse:.4f}")
print(f"✅ Test RMSE: {test_rmse:.4f}")
print(f"✅ Test MAE: {test_mae:.4f}")
print(f"✅ Test R²: {test_r2:.4f}")


In [ ]:
## 📊 Model Karşılaştırması

results_df = pd.DataFrame(results)

print("=" * 100)
print("📊 MODEL KARŞILAŞTIRMA SONUÇLARI")
print("=" * 100)
print("\n" + results_df.to_string(index=False))

# En iyi modeli bul
best_model_idx = results_df['Test_RMSE'].idxmin()
best_model = results_df.loc[best_model_idx]

print("\n" + "=" * 100)
print("🏆 EN İYİ MODEL")
print("=" * 100)
print(f"Model: {best_model['Model']}")
print(f"Best Params: {best_model['Best_Params']}")
print(f"CV RMSE: {best_model['CV_RMSE_Mean']:.4f}")
print(f"Test RMSE: {best_model['Test_RMSE']:.4f}")
print(f"Test MAE: {best_model['Test_MAE']:.4f}")
print(f"Test R²: {best_model['Test_R2']:.4f}")

# Görselleştirme
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RMSE karşılaştırması
axes[0].bar(results_df['Model'], results_df['Test_RMSE'], color='skyblue')
axes[0].set_title('Test RMSE Karşılaştırması', fontweight='bold')
axes[0].set_ylabel('RMSE')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# MAE karşılaştırması
axes[1].bar(results_df['Model'], results_df['Test_MAE'], color='lightcoral')
axes[1].set_title('Test MAE Karşılaştırması', fontweight='bold')
axes[1].set_ylabel('MAE')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

# R² karşılaştırması
axes[2].bar(results_df['Model'], results_df['Test_R2'], color='lightgreen')
axes[2].set_title('Test R² Karşılaştırması', fontweight='bold')
axes[2].set_ylabel('R² Score')
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
